In [7]:
!pip install crewai-tools

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 714.5/714.5 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 41.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 34.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 46.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 786.2/786.2 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 996.7/996.7 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 43.0 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [29]:
from dotenv import load_dotenv
load_dotenv()
import os
import requests
from typing import Type
from crewai.tools import BaseTool
from pydantic import BaseModel, Field

class CurrencyConverterInput(BaseModel):
    """Input model for CurrencyConverterTool"""
    amount: float = Field(..., description="The amount of currency to convert")
    from_currency: str = Field(..., description="The currency to convert from")
    to_currency: str = Field(..., description="The currency to convert to")

class CurrencyConverterTool(BaseTool):
    name: str = "Currency Converter"
    description: str = "Convert between different currencies using real-time exchange rates"
    input_schema: Type[BaseModel] = CurrencyConverterInput
    api_key: str = os.getenv("EXCHANGE_RATE_API_KEY")
    
    
    def _run(self, amount: float, from_currency: str, to_currency: str) -> str:
        url=f"https://v6.exchangerate-api.com/v6/{self.api_key}/latest/{from_currency}"
        response=requests.get(url)
        if response.status_code!= 200:
            return "Failed to fetch exchange rates"
        data=response.json()
        if "conversion_rates" not in data or to_currency not in data["conversion_rates"]:
            return f"Invalid currency code: {to_currency}"
        rate = data["conversion_rates"][to_currency]
        converted_amount = amount * rate
        return f"{amount} {from_currency} is equivalent to {converted_amount:.2f} {to_currency}"
        
from crewai import Agent
currency_analyst = Agent(
    role="Currency Analyst",
    goal="Convert the given amount of currency to the target currency",
    backstory=(
    "You are a currency analyst with a deep understanding of the foreign exchange market. "
    "You are tasked with converting the given amount of currency to the target currency."
    ),
    tools=[CurrencyConverterTool()],
    verbose=True

)
from crewai import Task

currency_task = Task(
    description=(
        "Convert {amount} {from_currency} to {to_currency}"
        "using real time exchange rates"
        "provide the equivalent amount in the target currency"
        "explain the relevant financial details"
    
    
    ),
    expected_output=(
        "The equivalent amount in the target currency"
        "The relevant financial details"
    ),
    agent=currency_analyst,

)
from crewai import Crew,Process

crew = Crew(
    agents=[currency_analyst],
    tasks=[currency_task],
    process=Process.sequential,
)
response=crew.kickoff(inputs={
        "amount": 100,
        "from_currency": "USD",
        "to_currency": "INR"})
print(response)

╭────────────────────────────── 🤖 Agent Started ──────────────────────────────╮
│                                                                              │
│  Agent: Currency Analyst                                                     │
│                                                                              │
│  Task: Convert 100 USD to INRusing real time exchange ratesprovide the       │
│  equivalent amount in the target currencyexplain the relevant financial      │
│  details                                                                     │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────── 🔧 Agent Tool Execution ───────────────────────────╮
│                                                                              │
│  Agent: Currency Analyst                                                     │
│                            